## import usefull library

In [1]:
import tensorflow as tf
import cv2
import h5py
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Activation, Dropout, Conv2D, Flatten, MaxPooling2D

## load the dataset

In [2]:
train_dataset = h5py.File('data/mask_data.h5')
X_data = np.array(train_dataset['X_data'][:])
Y_data = np.array(train_dataset['Y_data'][:])


## normalize the dataset

In [3]:
X_data = X_data / 255.0

## shuffle the data

In [4]:
idx = [i for i in range(X_data.shape[0])]
idx = np.random.permutation(idx) 
X_data = X_data[idx,:]
Y_data = Y_data[idx]

print("X_data shape: ", str(X_data.shape))
print("Y_data shepa: ", str(Y_data.shape))


X_data shape:  (1376, 128, 128, 3)
Y_data shepa:  (1376,)


## split the data in train set and test set
80% train set, 20% test set

In [5]:
i = int(X_data.shape[0] * .8)
X_train = X_data[:i,:]
Y_train = Y_data[:i]
X_test = X_data[i:,:]
Y_test = Y_data[i:]

print("X_train shape: ", str(X_train.shape))
print("Y_train shape: ", str(Y_train.shape))
print("X_test shape: ", str(X_test.shape))
print("Y_test shape: ", str(Y_test.shape))

X_train shape:  (1100, 128, 128, 3)
Y_train shape:  (1100,)
X_test shape:  (276, 128, 128, 3)
Y_test shape:  (276,)


In [10]:
image_size = X_train.shape[1]

model = Sequential()
model.add(Conv2D(200,(3,3),input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(1,activation='softmax'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 126, 200)     5600      
_________________________________________________________________
activation_4 (Activation)    (None, 126, 126, 200)     0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 200)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 61, 61, 100)       180100    
_________________________________________________________________
activation_5 (Activation)    (None, 61, 61, 100)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 100)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 90000)            

In [11]:
history = model.fit(X_train,Y_train,epochs = 20, validation_split = 0.2)

Epoch 1/20
 1/28 [>.............................] - ETA: 3s - loss: 8.1012 - accuracy: 0.4688

In [ ]:
faceCascade=cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)  #starts the webcam
labels_dict = {0:'NO MASK',1:'MASK'}
color_dict  = { 0:(0,0,255),1:(0,255,0)}

while(True):
    ret,frame = video_capture.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,1.3,5)
    
    for x,y,w,h in faces:
        face_img = gray[y:y+w,x:x+h]
        #resized = cv2.resize(face_img,(img_size,img_size))
        #normalized = resized/255.0
        #reshaped = np.reshape(normalized,(1,img_size,img_size,1))
        #result = model.predict(reshaped)
        
        #label = np.argmax(result,axis=1)[0]
        #cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        #cv2.putText(frame,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[1],2)
        cv2.putText(frame,labels_dict[1],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('Video',frame)
    key=cv2.waitKey(1)
    
    if(key==27):
        break;
        
cv2.destroyAllWindows()
video_capture.release()